In [1]:
import sys
from pathlib import Path

src_path = Path.cwd().parent / "src"
sys.path.append(str(src_path))

import json
import types
from collections import OrderedDict
from pprint import pprint

import torch
import torch.nn as nn
from dataset import get_new_dataloader
from funcs import get_dataloaders
from models import PreActResNet18, WideResNet
from train import epoch, epoch_test


In [2]:
# constant
CWD_PATH = Path.cwd()
MODEL_DIR_PATH = CWD_PATH.parent / "models/CIFAR10-diff-norm-value-cossim-01-fp"
SETTINGS = [
    {
        "normalization_mean": [-0.631, -0.97, -0.058],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.365, -0.892, -0.559],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.668, -0.004, 0.165],
        "normalization_std": None,
    },
    {
        "normalization_mean": [-0.631, -0.648, 0.624],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.598, 0.036, -0.537],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.78, 0.026, -0.51],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.648, -0.572, 0.483],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.26, 0.855, -0.536],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.847, -0.447, 0.64],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.456, 0.837, 0.251],
        "normalization_std": None,
    },
]


In [3]:
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_independent__mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__std_{std_str}"
    else:
        label = f"model__baseline"

    print(label)

model__mean_-0.631_-0.97_-0.058
model__mean_0.365_-0.892_-0.559
model__mean_-0.668_-0.004_0.165
model__mean_-0.631_-0.648_0.624
model__mean_0.598_0.036_-0.537
model__mean_0.78_0.026_-0.51
model__mean_0.648_-0.572_0.483
model__mean_0.26_0.855_-0.536
model__mean_0.847_-0.447_0.64
model__mean_0.456_0.837_0.251


In [4]:
def load_model(
    model_type: str,
    location: str,
    model_normalize: bool,
):
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")

    # load model CIFAR10
    if model_type == "pre-act-18":
        model = PreActResNet18(num_classes=10, normalize=model_normalize)
    elif model_type == "independent":
        model = WideResNet(
            n_classes=10,
            depth=16,  # deep_full for CIFAR10
            widen_factor=1,
            normalize=model_normalize,
            dropRate=0.3,
        )
    else:
        model = WideResNet(
            n_classes=10,
            depth=28,  # deep_full for CIFAR10
            widen_factor=10,
            normalize=model_normalize,
            dropRate=0.3,
        )

    try:
        model = model.to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    except:
        model = nn.DataParallel(model).to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    model.eval()
    return model


In [5]:
model_name__tr_acc = OrderedDict()

# model performance in TR
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_independent__mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__std_{std_str}"
    else:
        label = f"model__baseline"

    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "independent"
    args.experiment = 'diff-norm-value'
    args.dataset = 'CIFAR10'
    args.batch_size = 1000
    args.num_workers = 8
    args.normalization_mean = normalization_mean
    args.normalization_std = normalization_std
    args.extra_preprocessing_type = ""
    train_loader, test_loader = get_new_dataloader(args=args)

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__tr_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }


/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-01-fp/model_independent__mean_-0.631_-0.97_-0.058/final.pt
Custom_normalization: Normalize(mean=[-0.631, -0.97, -0.058], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:27<00:00,  1.79it/s]


0.70886
1.0
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-01-fp/model_independent__mean_0.365_-0.892_-0.559/final.pt
Custom_normalization: Normalize(mean=[0.365, -0.892, -0.559], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.14it/s]


0.71522
1.0
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-01-fp/model_independent__mean_-0.668_-0.004_0.165/final.pt
Custom_normalization: Normalize(mean=[-0.668, -0.004, 0.165], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


...

0.69668
1.0
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-01-fp/model_independent__mean_0.847_-0.447_0.64/final.pt
Custom_normalization: Normalize(mean=[0.847, -0.447, 0.64], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.14it/s]


0.69196
1.0
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-01-fp/model_independent__mean_0.456_0.837_0.251/final.pt
Custom_normalization: Normalize(mean=[0.456, 0.837, 0.251], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.13it/s]


0.68988
1.0


In [6]:
pprint(model_name__tr_acc)

# Serializing json
json_object = json.dumps(model_name__tr_acc, indent=2)
 
# Writing to sample.json
with open("tr_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model_independent__mean_-0.631_-0.97_-0.058',
              {'test_acc': 1.0, 'train_acc': 0.70886}),
             ('model_independent__mean_0.365_-0.892_-0.559',
              {'test_acc': 1.0, 'train_acc': 0.71522}),
             ('model_independent__mean_-0.668_-0.004_0.165',
              {'test_acc': 1.0, 'train_acc': 0.70286}),
             ('model_independent__mean_-0.631_-0.648_0.624',
              {'test_acc': 1.0, 'train_acc': 0.69852}),
             ('model_independent__mean_0.598_0.036_-0.537',
              {'test_acc': 1.0, 'train_acc': 0.69204}),
             ('model_independent__mean_0.78_0.026_-0.51',
              {'test_acc': 1.0, 'train_acc': 0.68854}),
             ('model_independent__mean_0.648_-0.572_0.483',
              {'test_acc': 1.0, 'train_acc': 0.70222}),
             ('model_independent__mean_0.26_0.855_-0.536',
              {'test_acc': 1.0, 'train_acc': 0.69668}),
             ('model_independent__mean_0.847_-0.447_0.64',
            

In [8]:
model_name__te_acc = OrderedDict()

# model performance in FE
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_independent__mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model_independent__mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__std_{std_str}"
    else:
        label = f"model__baseline"

    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    train_loader, test_loader = get_dataloaders(
        dataset="CIFAR10",
        batch_size=1000,
        pseudo_labels=False,
        normalize=True,
        train_shuffle=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__te_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }



/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-01-fp/model_independent__mean_-0.631_-0.97_-0.058/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:29<00:00,  1.70it/s]


0.10084
0.1018
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-01-fp/model_independent__mean_0.365_-0.892_-0.559/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.61it/s]


0.11156
0.1115
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-01-fp/model_independent__mean_-0.668_-0.004_0.165/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


...

0.09898
0.0988
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-01-fp/model_independent__mean_0.847_-0.447_0.64/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.60it/s]


0.13628
0.1386
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-cossim-01-fp/model_independent__mean_0.456_0.837_0.251/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.60it/s]


0.15484
0.1652


In [9]:
pprint(model_name__te_acc)

# Serializing json
json_object = json.dumps(model_name__te_acc, indent=2)
 
# Writing to sample.json
with open("fe_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model_independent__mean_-0.631_-0.97_-0.058',
              {'test_acc': 0.1018, 'train_acc': 0.10084}),
             ('model_independent__mean_0.365_-0.892_-0.559',
              {'test_acc': 0.1115, 'train_acc': 0.11156}),
             ('model_independent__mean_-0.668_-0.004_0.165',
              {'test_acc': 0.1017, 'train_acc': 0.10122}),
             ('model_independent__mean_-0.631_-0.648_0.624',
              {'test_acc': 0.1004, 'train_acc': 0.10042}),
             ('model_independent__mean_0.598_0.036_-0.537',
              {'test_acc': 0.149, 'train_acc': 0.14144}),
             ('model_independent__mean_0.78_0.026_-0.51',
              {'test_acc': 0.1813, 'train_acc': 0.16488}),
             ('model_independent__mean_0.648_-0.572_0.483',
              {'test_acc': 0.1406, 'train_acc': 0.13958}),
             ('model_independent__mean_0.26_0.855_-0.536',
              {'test_acc': 0.0988, 'train_acc': 0.09898}),
             ('model_independent__mean_0.847_-0.